In [ ]:
# ***************************************************************
# Preprocess Microsoft Academics Graph (MAG) Dataset
# ***************************************************************

# Jupyter Notebook for the preprocessing of the Microsoft Academics Graph (MAG) dump
#
# TODO **********
# In particular, the following operations are going to be executed:
# - Opening of ConferenceInstances and ConferenceSeries CSVs
# - Drop of the useless columns 
# - Merge of the two CSVs on the ConferenceSeriesID column
# - Chuncked Processing of the Papers CSV
# ---- Drop of the useless columns
# ---- Drop of papers from journals and books rows
# - Merge with the processed conferences data
#
# Lastly, the entire preprocessed dump is going to be saved on disk in CSV format

In [ ]:
# Libraries Import
import pandas as pd

pd.set_option('display.max_columns', None)

In [ ]:
# ******************* PATHS ********************+

# Dumps Directory Path
path_file_import = r'/Users/marcoterzulli/File/Scuola Local/Magistrale/Materiale Corsi Attuali/Tirocinio/Cartella di Lavoro/Archivi Dump di Lavoro/MAG/'

# CSV Exports Directory Path
path_file_export = r'/Users/marcoterzulli/File/Scuola Local/Magistrale/Materiale Corsi Attuali/Tirocinio/Cartella di Lavoro/Archivi Dump di Lavoro/Export/MAG_Chunks/'

In [ ]:
# ******************* CONFERENCE INSTANCES ********************

# Read of the Conference Instances File

# The column names follow the MAG' scheme official documentation
df_mag_conf_instances_col_names = ['ConferenceInstanceID', 'NormalizedName', 'DisplayName', 'ConferenceSeriesID', 'Location', 'OfficialUrl', 'StartDate', 'EndDate', 'AbstractRegistrationDate', 'SubmissionDeadlineDate', 'NotificationDueDate', 'FinalVersionDueDate', 'PageCount', 'PaperFamilyCount', 'CitationCount', 'Latitude', 'Longitude', 'CreatedDate']

df_mag_conf_instances = pd.read_csv(path_file_import + 'ConferenceInstances.txt', sep='\t', names=df_mag_conf_instances_col_names)
df_mag_conf_instances

In [ ]:
# Drop of Conference Instances' Useless Columns
df_mag_conf_instances = df_mag_conf_instances.drop(columns=['OfficialUrl', 'AbstractRegistrationDate', 'SubmissionDeadlineDate', 'NotificationDueDate', 'FinalVersionDueDate', 'PageCount', 'PaperFamilyCount', 'CitationCount', 'Latitude', 'Longitude', 'CreatedDate', 'StartDate', 'EndDate'])
df_mag_conf_instances

In [ ]:
# Column rename to remove ambiguity for the future joins
df_mag_conf_instances.rename(columns={'NormalizedName': 'ConferenceNormalizedName', 'DisplayName': 'ConferenceDisplayName', 'Location': 'ConferenceLocation'}, inplace=True)
df_mag_conf_instances

In [ ]:
# ******************* CONFERENCE SERIES ********************

# Read of the Conference Series File

# The column names follow the MAG' scheme official documentation
df_mag_conf_series_col_names = ['ConferenceSeriesID', 'Rank', 'NormalizedName', 'DisplayName', 'PaperCount', 'PaperFamilyCount', 'CitationCount', 'CreatedDate']

df_mag_conf_series = pd.read_csv(path_file_import + 'ConferenceSeries.txt', sep='\t', names=df_mag_conf_series_col_names)
df_mag_conf_series

In [ ]:
# Drop of Conference Series' Useless Columns
df_mag_conf_series = df_mag_conf_series.drop(columns=['Rank', 'PaperCount', 'PaperFamilyCount', 'CitationCount', 'CreatedDate'])
df_mag_conf_series

In [ ]:
# Column rename to remove ambiguity for the future joins
df_mag_conf_series.rename(columns={'NormalizedName': 'ConferenceSeriesNormalizedName', 'DisplayName': 'ConferenceSeriesDisplayName'}, inplace=True)
df_mag_conf_series

In [ ]:
# ******************* MERGE OF CONFERENCE DATA ********************

# Merge of the conference series and conference instances dataframes over the conferenceseriesid column
df_mag_conf_merged = df_mag_conf_instances.merge(df_mag_conf_series, on='ConferenceSeriesID')
df_mag_conf_merged

In [ ]:
# ******************* PAPERS ********************

# The Papers CSV is going to be processed in chunks, due to its size

# The column names follow the MAG' scheme official documentation
df_mag_papers_col_names = ['PaperID', 'Rank', 'Doi', 'DocType', 'PaperTitle', 'OriginalTitle', 'BookTitle', 'Year', 'Date', 'OnlineDate', 'Publisher', 'JournalID', 'ConferenceSeriesID', 'ConferenceInstanceID', 'Volume', 'Issue', 'FirstPage', 'LastPage', 'ReferenceCount', 'CitationCount', 'EstimatedCitation', 'OriginalVenue', 'FamilyID', 'FamilyRank', 'Retracion', 'CreatedDate']

# List of processed chunks.
df_mag_papers_list_of_chunks = list()

# Define of the chunk size
chunksize = 10 ** 7

count = 1
with pd.read_csv(path_file_import + 'Papers.txt', sep='\t', chunksize=chunksize, low_memory=False, on_bad_lines='skip', names=df_mag_papers_col_names) as reader:
    for chunk in reader:

        # Drop of the useless columns
        chunk = chunk.drop(columns=['Rank', 'OnlineDate', 'Publisher', 'Volume', 'Issue', 'FirstPage', 'LastPage', 'ReferenceCount', 'OriginalVenue', 'FamilyID', 'FamilyRank', 'Retracion', 'CreatedDate', 'JournalID', 'BookTitle'])

        # Filtering of papers without DOI
        chunk = chunk.dropna(subset = ['Doi'])

        # Filtering papers that are not related to conferences
        chunk = chunk[chunk.DocType == 'Conference']

        # Drop of the doctype column
        chunk = chunk.drop(columns=['DocType'])

        # Writing the partial CSV on Disk
        chunk.to_csv(path_file_export + 'out_mag_citations_chunk_'+ str(count) +'.csv')

        print(f'Successfully processed chunk {count} out of around {200000000 / chunksize}')
        count += 1

# Concatenation of the processed chunks
#df_mag_papers = pd.concat(df_mag_papers_list_of_chunks)

# Empty the list to free some memory
#df_mag_papers_list_of_chunks = list()

In [ ]:
chunk

In [ ]:
# ******************* PAPERS ********************

# The Papers CSV is going to be processed in chunks, due to its size

# The column names follow the MAG' scheme official documentation
df_mag_papers_col_names = ['PaperID', 'Rank', 'Doi', 'DocType', 'PaperTitle', 'OriginalTitle', 'BookTitle', 'Year', 'Date', 'OnlineDate', 'Publisher', 'JournalID', 'ConferenceSeriesID', 'ConferenceInstanceID', 'Volume', 'Issue', 'FirstPage', 'LastPage', 'ReferenceCount', 'CitationCount', 'EstimatedCitation', 'OriginalVenue', 'FamilyID', 'FamilyRank', 'CreatedDate']

# List of processed chunks.
df_mag_papers_list_of_chunks = list()

# Define of the chunk size
chunksize = 10 ** 7

count = 1
with pd.read_csv(path_file_import + 'Papers.txt', sep='\t', names=df_mag_papers_col_names, chunksize=chunksize, low_memory=False, on_bad_lines='skip') as reader:
    for chunk in reader:
        print(f'Currently processing chunk {count} out of around 400')
        count += 1

        # Drop of the useless columns
        chunk = chunk.drop(columns=['Rank', 'OnlineDate', 'Publisher', 'Volume', 'Issue', 'FirstPage', 'LastPage', 'ReferenceCount', 'OriginalVenue', 'FamilyID', 'FamilyRank', 'CreatedDate'])

        # Filtering of books and Journals papers
        # TODO

        # Insert of the resulting chunk in the list 
        df_mag_papers_list_of_chunks.append(chunk)

# Concatenation of the processed chunks
df_mag_papers = pd.concat(df_mag_papers_list_of_chunks)

# Empty the list to free some memory
df_mag_papers_list_of_chunks = list()

In [ ]:
df_mag_papers

In [ ]:
# Export of the partial dataframe
df_mag_papers.to_csv(path_file_export + 'out_mag_citations_partial.csv')
print(f'Successfully Exported the Partial Preprocessed CSV to {path_file_export}out_mag_citations_partial.csv')


In [ ]:
# Get All Files' Names
coci_all_csvs = glob.glob(path_file_import + "*.csv")

In [ ]:
df_coci_processed = pd.DataFrame(columns=['article', 'citations_count'])

# Combine new data with a partial CSV
if combine_with_partial_csv:
    df_coci_processed = pd.read_csv(partial_csv_path, low_memory=False)
    print(f'Successfully Imported the Partial CSV')
    
# Read, process and concat all CSVs
count = 0
for current_csv_name in coci_all_csvs:

    # Open the current CSV
    print(f'Currently processing CSV {count}: {current_csv_name}')
    count += 1
    df_coci_current_csv = pd.read_csv(current_csv_name, low_memory=False)

    # Drop of the useless columns: 'oci', 'citing', 'creation', 'timespan', 'journal_sc', 'author_sc'
    df_coci_current_csv.drop(columns=['oci', 'citing', 'creation', 'timespan', 'journal_sc', 'author_sc'])

    # Group by cited article and count
    sf_coci_current_grouped = df_coci_current_csv.groupby(['cited'])['cited'].count()

    # Since the returned object is a Pandas Series type, we need to convert it to a Pandas dataframe
    df_coci_current_csv = pd.DataFrame({'article':sf_coci_current_grouped.index, 'citations_count':sf_coci_current_grouped.values})

    ### Concat with the data previously elaborated
    df_coci_processed = pd.concat([df_coci_processed, df_coci_current_csv])

    # Now we need to do a new group by and sum the citations_count to reduce the data
    sf_coci_processed_grouped = df_coci_processed.groupby(['article'])['citations_count'].sum()
    df_coci_processed = pd.DataFrame({'article':sf_coci_processed_grouped.index, 'citations_count':sf_coci_processed_grouped.values})

# Export of the final dataframe
df_coci_processed.to_csv(path_file_export + 'out_coci_citations_count.csv')
print(f'Successfully Exported the Preprocessed CSV to {path_file_export}out_coci_citations_count.csv')


In [ ]:
# Check of the Exported CSV
df_coci_exported_csv = pd.read_csv(path_file_export + 'out_coci_citations_count.csv', low_memory=False)
df_coci_exported_csv

In [ ]:
# Order by citations count descending to see the articles with the most citations
df_coci_exported_csv = df_coci_exported_csv.sort_values(by='citations_count', ascending=False)
df_coci_exported_csv

In [ ]:
# Checking the total count of the citations contained in the extracted CSV
df_coci_exported_csv['citations_count'].sum()